In [100]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import math

print(os.getcwd())
def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def region_of_interest(img):
    mask = np.zeros_like(img)
    
    if len(img.shape) > 2:
        channel_count = img.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    x_dim = img.shape[1]   
    y_dim = img.shape[0]
    #vertices = np.array([[int((x_dim)/2),int((y_dim)/2)],[int((x_dim)/6),int(y_dim) - 1],[int((x_dim)*7/8),int(y_dim) - 1]])   
    vertices = np.array([(int((x_dim)*5/11),int((y_dim)*3/5)),(int((x_dim)/6),int(y_dim) - 1),(int((x_dim)*7/8),int(y_dim) - 1),(int((x_dim)*6/11),int((y_dim)*3/5))])   
    cv2.fillPoly(mask, [vertices], ignore_mask_color)
    #cv2.imshow("Mask", mask)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255,0,0], thickness=5):
    for line in lines:
        cv2.line(img, (line[0][0], line[0][1]), (line[0][2], line[0][3]), color, thickness)
        
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines

def interpolate_line(lines, ysize, xsize):
    left_lines = []
    right_lines = []
    final_lines = []
    for line in lines:
        if ((line[0][1] - line[0][3])/(line[0][0] - line[0][2]) <= 0):
            left_lines.append(line)
        else:
            right_lines.append(line)
    average_slope = 0
    average_intercept = 0
    for line in left_lines:
        slope = ((line[0][1] - line[0][3]) / (line[0][0] - line[0][2]))
        if (np.isfinite(((line[0][1] - line[0][3]) / (line[0][0] - line[0][2])))):
            average_slope += slope 
            average_intercept += line[0][1] - slope * line[0][0] 
    average_slope = average_slope / len(left_lines)
    print("Aditya", average_slope)
    average_intercept = average_intercept / len(left_lines)
    x_intercept = int((ysize - 1 - average_intercept)/ average_slope)
    x_intercept_1 = int((ysize * 6 / 10 - average_intercept)/ average_slope)
    final_lines.append(np.array([[x_intercept_1, int(ysize * 6 / 10), x_intercept, ysize - 1]]))
    
    #############################################################################################
    
    average_slope = 0
    average_intercept = 0
    if (len(right_lines) <= 0):
        return final_lines
    for line in right_lines:
        if (np.isfinite(((line[0][1] - line[0][3]) / (line[0][0] - line[0][2])))):
            slope = ((line[0][1] - line[0][3]) / (line[0][0] - line[0][2]))
            average_slope += slope 
            average_intercept += line[0][1] - slope * line[0][0]
    average_slope = average_slope / len(right_lines)
    average_intercept = average_intercept / len(right_lines)
    x_intercept = int((ysize - 1 - average_intercept)/ average_slope)
    x_intercept_1 = int((ysize * 6 / 10 - average_intercept)/ average_slope)
    final_lines.append(np.array([[x_intercept_1, int(ysize * 6 / 10), x_intercept, ysize - 1]]))
    print("Zeher", final_lines, average_slope)
    return final_lines
    
def process_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur_gray = gaussian_blur(gray, 3)
    canny_blur_gray = canny(blur_gray, 50, 150)
    region_canny_blur_gray = region_of_interest(canny_blur_gray)
    lines = hough_lines(region_canny_blur_gray, 1, np.pi/180, 30, 1, 10)
    lines = interpolate_line(lines, image.shape[0], image.shape[1])
    blank_img = np.zeros_like(image)
    draw_lines(blank_img, lines)
    image = cv2.addWeighted(image, 0.8, blank_img, 1, 0)
    #draw_lines(image, lines)
    return image
#cv2.imshow("Output1 image", image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

C:\Users\aditya.singla\sdc_proj\CarND-LaneLines-P1




                                                                                                                 
                                                                                                                       


                                                                                                              

t:   2%|█▌                                                                 | 5/221 [1:18:23<00:17, 12.64it/s, now=None]


t:   6%|███▉                                                                | 40/681 [20:40<00:15, 42.17it/s, now=None]

                                                                                                                 
                                                                                                                       


                                                                                                              

t:   2%|█▌                                                 

Aditya -0.7059841280591612
Zeher [array([[458, 324, 153, 539]]), array([[510, 324, 854, 539]])] 0.6256410256410256
Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4

Aditya -0.7059841280591612
Zeher [array([[458, 324, 153, 539]]), array([[510, 324, 854, 539]])] 0.6256410256410256






t:   1%|▌                                                                    | 2/221 [00:00<00:14, 15.04it/s, now=None]



t:   4%|██▍                                                                  | 8/221 [00:00<00:11, 19.31it/s, now=None]

Aditya -0.709111012826183
Zeher [array([[454, 324, 150, 539]]), array([[510, 324, 860, 539]])] 0.615214043215705
Aditya -0.7340698312589532
Zeher [array([[452, 324, 159, 539]]), array([[509, 324, 854, 539]])] 0.6233422940688079
Aditya -0.7276026818709745
Zeher [array([[453, 324, 158, 539]]), array([[511, 324, 853, 539]])] 0.6280261156489512
Aditya -0.7225574878636103
Zeher [array([[452, 324, 154, 539]]), array([[511, 324, 856, 539]])] 0.6228828463203464
Aditya -0.7306870580439914
Zeher [array([[450, 324, 156, 539]]), array([[510, 324, 856, 539]])] 0.6211676741193892
Aditya -0.7604029956971133
Zeher [array([[442, 324, 159, 539]]), array([[512, 324, 855, 539]])] 0.6260494235053923
Aditya -0.7248187211601846
Zeher [array([[451, 324, 155, 539]]), array([[511, 324, 862, 539]])] 0.6125652875603319
Aditya -0.6748914451330088
Zeher [array([[456, 324, 137, 539]]), array([[510, 324, 854, 539]])] 0.6260795130279402
Aditya -0.7200713618780846
Zeher [array([[454, 324, 156, 539]]), array([[509, 324,

...

 0.5615615615615616
Aditya -0.7790858480655288
Zeher [array([[454, 324, 178, 539]]), array([[514, 324, 895, 539]])] 0.5638176638176638
Aditya -0.7754302815683889
Zeher [array([[458, 324, 181, 539]]), array([[512, 324, 900, 539]])] 0.5539175539175539
Aditya -0.7242063492063492
Zeher [array([[460, 324, 163, 539]]), array([[512, 324, 903, 539]])] 0.549604916593503
Aditya -0.7616129915073185
Zeher [array([[458, 324, 176, 539]]), array([[510, 324, 893, 539]])] 0.5614325842696629
Aditya -0.7950846394276185
Zeher [array([[456, 324, 186, 539]]), array([[510, 324, 888, 539]])] 0.569451934977563
Aditya -0.7939662930952129
Zeher [array([[457, 324, 186, 539]]), array([[507, 324, 891, 539]])] 0.560234408481935
Aditya -0.7885394359311473
Zeher [array([[458, 324, 185, 539]]), array([[508, 324, 886, 539]])] 0.5688643867748345
Aditya -0.8053359414133855
Zeher [array([[457, 324, 190, 539]]), array([[507, 324, 882, 539]])] 0.572115464868712
Aditya





t:  99%|██████████████████████████████████████████████████████████████████▍| 219/221 [00:04<00:00, 44.35it/s, now=None]



                                                                                                                       

 -0.7763439866529332
Zeher [array([[458, 324, 181, 539]]), array([[508, 324, 884, 539]])] 0.5714202084065099
Aditya -0.7804761947345068
Zeher [array([[457, 324, 181, 539]]), array([[507, 324, 823, 539]])] 0.6805561929763368
Aditya -0.7636287751656028
Zeher [array([[459, 324, 177, 539]]), array([[510, 324, 891, 539]])] 0.5642981086239806
Aditya -0.772536760182109
Zeher [array([[458, 324, 179, 539]]), array([[509, 324, 883, 539]])] 0.5750938930904975
Aditya -0.7697671279187012
Zeher [array([[458, 324, 178, 539]]), array([[509, 324, 887, 539]])] 0.5699958331537279




                                                                                                                 
                                                                                                                       


                                                                                                              

t:   2%|█▌                                                                 | 5/221 [1:18:29<00:17, 12.64it/s, now=None]


t:   6%|███▉                                                                | 40/681 [20:45<00:15, 42.17it/s, now=None]

                                                                                                                 
                                                                                                                       


                                                                                                              

t:   2%|█▌                                                 

Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
Wall time: 5.71 s
